## $P^{n}(X_{4})$ の位数

$$
P^{n}(X_{4}) = \{(\sigma_{1}, \dots, \sigma_{n}) | \sigma_{1}\sigma_{2} \cdots \sigma_{n} = 1, \sigma_{i} \neq \sigma_{j} [\forall i,j] \}
$$

In [1]:
require 'objspace'; nil

## Cyclic permutations

In [2]:
def act(sigma, k)
  if k.is_a? Array
    return k.map{|v| act(sigma, v)}
  else
    ind = (sigma.index(k)+1).then{|x| (x == 4) ? 0 : x}
    return sigma[ind]
  end
end; nil

In [3]:
a = (0..3).to_a
sigma = [2,3,0,1]
tau = [3,2,0,1]
sa = act(sigma, a)
tsa = act(tau, sa)

puts "#{a}   |-#{sigma}->   #{sa}   |-#{tau}->   #{tsa}"
nil

[0, 1, 2, 3]   |-[2, 3, 0, 1]->   [1, 2, 3, 0]   |-[3, 2, 0, 1]->   [3, 0, 2, 1]


## Tuples of cyclic permutations

In [4]:
k = 4
PMLIST = (0..k-1).to_a.permutation.to_a
CPLIST = PMLIST.select{|p| p.first == 0} #(0..k-2).to_a.permutation.map{|s| s << (k-1)}

INITIAL = (0..k-1).to_a
EVENS = (0..100).map{|k| 2*k}

#p PMLIST, CPLIST
nil

In [5]:
CPLIST

[[0, 1, 2, 3], [0, 1, 3, 2], [0, 2, 1, 3], [0, 2, 3, 1], [0, 3, 1, 2], [0, 3, 2, 1]]

In [6]:
sigma = CPLIST[0] #[2,3,0,1]
tau = CPLIST[1] #[3,2,0,1] 

si = PMLIST.index(act(sigma, INITIAL))
tsi = PMLIST.index(act(tau, PMLIST[si]))

puts "#{INITIAL}   |-#{sigma}->   #{PMLIST[si]}   |-#{tau}->   #{PMLIST[tsi]}"
nil

[0, 1, 2, 3]   |-[0, 1, 2, 3]->   [1, 2, 3, 0]   |-[0, 1, 3, 2]->   [3, 0, 2, 1]


In [7]:
class Tree
  def initialize(pm_ind, cp_ind, depth)
    @pm_ind = pm_ind
    @cp_ind = cp_ind
    @depth = depth
    @children = (@depth > 0 && self.value == INITIAL) ? nil : []
  end
  attr_accessor :depth, :children
  
  def value
    return PMLIST[@pm_ind]
  end
  def cp
    return @cp_ind.nil? ? nil : CPLIST[@cp_ind]
  end
  def dead?
    @children.nil?
  end
#-----
  
  def nodes(dp)
    if @depth == dp
      return [self]
    else 
      return (@children.nil?) ? [] : @children.map{|c| c.nodes(dp)}.flatten
    end
  end
  def terminals
    if @children.nil? 
      return []
    else
      return (@children.empty?) ? [self] : @children.map(&:terminals).flatten
    end
  end
  
  def max_depth
    return self.terminals.map(&:depth).max
  end
  
  def grow
    if (not self.dead?) && @children.empty?
      (0..CPLIST.size-1).each do |cp_ind|
        v = act(CPLIST[cp_ind], self.value)
        pm_ind = PMLIST.index(v)
        @children << Tree.new(pm_ind, cp_ind, @depth+1)
      end
    end
    return self
  end
#-----
  
  def show
    return "< v: #{self.value}, cp: #{self.cp}, d: #{@depth} >"
  end
  def to_s
    self.show
  end
  def inspect
    self.show
  end
end; nil

In [8]:
def get_ms_of(obj)
  total = 0
  ObjectSpace.reachable_objects_from(obj).each do |o|
    total += ObjectSpace.memsize_of(o)
  end
  puts "#{total * 0.001 * 0.001} MB"
end; nil

In [ ]:
root = Tree.new(0, nil, 0)

max = 2*4

nodes = [root]
ends = [0]
max.times do |d|
  nodes.map!{|n| (n.grow).children.then{|x| (x.nil?) ? [] : x} }.flatten!
#  get_ms_of(nodes)
  ends << nodes.count{|n| n.dead?}.tap{|x| puts "depth=#{d+1}: #{x}"} if (d+1).even?  
end
nil

depth=2: 6
depth=4: 84
depth=6: 2712


In [10]:
seq = [1]

# seq << ends[1]*seq[0]
# seq << ends[2]*seq[0] + ends[1]*seq[1]
# seq << ends[3]*seq[0] + ends[2]*seq[1] + ends[1]*seq[2]
# seq << ends[4]*seq[0] + ends[3]*seq[1] + ends[2]*seq[2] + ends[1]*seq[3]

(ends.size - 1).times do |k|
  seq << (k+1).times.map{|i| ends[i+1]*seq[k+1-(i+1)] }.sum
end
seq

[1, 6, 120, 3936, 140160]

In [12]:
"#{ObjectSpace.memsize_of_all * 0.001 * 0.001} MB"

"211.517154 MB"

## Experiments

In [ ]:
puts ends
puts [0] + ends.map{|v| v*6}
puts [0, 0] + ends.map{|v| v*120}
puts [0, 0, 0] + ends.map{|v| v*(2712+504+720)}
nil